# Libs

In [ ]:
# Instaling the Lib for generation data fakes.
# !pip install Faker

# Necessary libs for this notebook
from faker import Faker
import random
from pyspark.dbutils import DBUtils
from faker.providers import internet
from datetime import datetime
import pandas as pd
import os

# Functions

In [ ]:
def func_rename(local, new_local):

    from datetime import datetime

    for file_name in dbutils.fs.ls(local):
        file_name = file_name[0] # just the name
        if file_name[-8:] == 'parquet': #if the final name end with ".parquet"
        DBUtils.fs.cp(file_name, new_local) # Move the parquet with another name from other place
        DBUtils.fs.rm(local, True) # Delete the folder
    return print(f'File in: {new_local}') 

In [ ]:
# Generation data

fake = Faker()
fake.add_provider(internet)

def data_generaton(connection_quantity,device,status):
    # Creating a empity list
    fake_records = []
    connection_velocity = [1,2,3,4,5,10,20,25,40,50,100,500,1000]

# Generation fake data list
    for _ in range(connection_quantity):
        new_record = {
            'Name': fake.name(),
            'Address': fake.address(),
            'Ip': fake.ipv4_private(),
            'Time Connection': datetime.now(),
            'Device': device,
            'Connection Velocity': random.choice(connection_velocity),
            'Status': status
            }
        fake_records.append(new_record)

# Creating a dataframe from generated data
df = pd.DataFrame.from_records(fake_records)

# Converting the pandas dataframe in spark dataframe
spark_df = spark.createDataFrame(df)

# Save on landing_zone
try:
    Local = f"dbfs:/FileStore/landing_zone/{device}/{status}/{status}_{datetime.now()}"
    spark_df.coalesce(1).write.format("parquet").mode("overwrite").save(Local)
    new_local = f"dbfs:/FileStore/landing_zone/{device}/{status}/{device}/{device}_{datetime.now()}.parquet"

    func_rename(Local,new_local)

    return print(f'Sucessfully generated = {connection_quantity} connections of {device} | Status: {status} | Local File: {Local}')

except:
    return print('Error! Function def_generation() is not deffined')

In [ ]:
def clean_dbfs(argument):
    switch_case = {
       'landing_zone': lambda: delete_directory(dbutils, "dbfs:/FileStore/landing_zone")
    }
    switch_case.get(argument, lambda: print('Not found!'))

In [ ]:
def clear_diretory_by_path(path_local):

    # Path of the directory to be deleted
    directory_path = path_local

    # Start the object DBUtils
    dbutils = DBUtils(spark)

    # Check if the directory exists
    if dbutils.fs.mkdirs(directory_path):
        # Delete the files within the directory
        dbutils.fs.rm(directory_path, recurse=True)
        print(f"Files on directory {directory_path} was deleted with success.")
    else:
        print(f"The directory {directory_path} not exists.")
    